# [Binary Classification : Food Recognition]
In this notebook, we will show how to use multi-layer CNN to build a binary food image classification project, to distinguish food images from nonfood images.

Before running any code, we do the following two steps:
1. Reset the runtime by going to **Runtime -> Reset all runtimes** in the menu above. 
2. Select GPU by going to **Runtime -> Change runtime type -> Hardware accelerator** in the menu above. 


## [Importing packages]

First, we will import some packages which will be used in our codes.
- `os` : for read files 
- `numpy`: for matrix computation 
- `matplotlib.pyplot`: for graph plotting and display
- `tensorflow`: the main deep learning toolbox
- `tf.keras.preprocessing.image.ImageDataGenerator`: for load data


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## [Data Loading]
- Food-5K   
This is a dataset containing 2500 food and 2500 non-food images, for the task of food/non-food classification. The whole dataset is divided in three parts: training, validation and evaluation. The naming convention is as follows:
{ClassID}_{ImageID}.jpg
ClassID: 0 or 1; 0 means non-food and 1 means food.
ImageID: ID of the image within the class.    
More info can be seen from [Food/Non-food Image Classification and Food Categorization using Pre-Trained GoogLeNet Model](https://infoscience.epfl.ch/record/221610/files/madima2016_food_recognition.pdf)

  The dataset is organized as follows:
- Food-5K
  - training   
    - food
    - non-food   
 - validation   
   - food
   - non-food
 - evaluation
   - food
   - non-food
          

In [0]:
## get_file has some problem with google drive or dropbox link
#_URL =  'https://www.dropbox.com/s/zco960drgtjokht/Food-5K.zip'
#zip_dir = tf.keras.utils.get_file('Food_5K.zip', origin=_URL, extract=True)

In [0]:
if not os.path.exists('/root/.keras/datasets/'):
    os.makedirs('/root/.keras/datasets/')

In [0]:
%cd /root/.keras/datasets/
!wget https://www.dropbox.com/s/zco960drgtjokht/Food-5K.zip
!unzip -uq Food-5K.zip

We'll now assign variables with the proper file path for the training and validation sets.


In [0]:
base_dir = '/root/.keras/datasets/Food-5K'
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'evaluation')

In [0]:
train_food_dir = os.path.join(train_dir, 'food')   # directory with our training food pictures
train_nonfood_dir = os.path.join(train_dir,'non-food')   # directory with our training non-food pictures
validation_food_dir = os.path.join(validation_dir, 'food')  
validation_nonfood_dir = os.path.join(validation_dir,'non-food')

test_food_dir =  os.path.join(test_dir, 'food')
test_nonfood_dir = os.path.join(test_dir,'non-food')


### Data exploration


In [0]:
num_food_tr = len(os.listdir(train_food_dir))
num_nonfood_tr = len(os.listdir(train_nonfood_dir))

num_food_val = len(os.listdir(validation_food_dir))
num_nonfood_val = len(os.listdir(validation_nonfood_dir))

total_train = num_food_tr + num_nonfood_tr
total_val = num_food_val + num_nonfood_val


num_food_test = len(os.listdir(test_food_dir))
num_nonfood_test = len(os.listdir(test_nonfood_dir))
total_test = num_food_test + num_nonfood_test

In [0]:
print('total training food images:', num_food_tr )
print('total training non-food images:', num_nonfood_tr)

print('total validation food images:', num_food_val)
print('total validation non-food images:', num_nonfood_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

### Define several model parameters
Here, we will set up variables that will be used later while pre-processing our dataset and training our network.

In [0]:
BATCH_SIZE = 30
IMG_SHAPE = 150

### Data preparation

Images must be formatted into appropriately pre-processed floating point tensors before being fed into the network. We use `tf.keras.preprocessing.image.ImageDataGenerator` to do data preparation.
 
- Read images from the disk
- Convert them into floating point tensors for usage in networks
- Rescale the tensors from values between 0 and 255 to values between 0 and 1.

In [0]:
datagen = ImageDataGenerator(rescale=1./255)  

`flow_from_directory` method can load images from the disk, and apply rescaling, and resize on images.

In [0]:
train_datagen = datagen.flow_from_directory(directory=train_dir, 
                                                           shuffle=True, 
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150) 
                                                           batch_size=BATCH_SIZE, 
                                                           class_mode='binary')
val_datagen = datagen.flow_from_directory(directory=validation_dir, 
                                                           shuffle=False, 
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150) 
                                                           batch_size=BATCH_SIZE, 
                                                           class_mode='binary')

Here, we show some image samples from the dataset.

In [0]:
# plots images with labels 
def plotImg(ims, figsize=(20,20),titles=None):  
    f = plt.figure(figsize=figsize)
    for i in range(12):
        sp = f.add_subplot(4, 6, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=15)
        plt.imshow(ims[i])
        
imgs, labels = next(train_datagen)
plotImg(imgs, titles=labels.astype(int))


## [Construct the Model]

We build a model with 3 convolution layers with max-pooling after each conv layer and 2 fully connected layer.
activation function -- 'relu' 

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

Here, we set up the optimizer and loss function.
We use `adam` optimizer,  `sparse_categorical_crossentropy` as the loss function, `accuracy` as the performance measure.

In [0]:
model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

We can see the model architecture by using the `summary` function.

In [0]:
model.summary()

## [Training Model]

In [0]:
EPOCHS = 20
history = model.fit_generator(
    train_datagen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_datagen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

### Visualize the training process   
Here, we visualize the training process to have a better understanding of our model.
To see the performance of our model, and get some insights on how to tune our model for better performance.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8,8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()

## [Evaluating Model]
It is always a good practice to evaludate our models on held out samples.
We use the training data to learn the neural networks parameters, such as the weights and bias.
We use the validation data to verify whether our model is overfitting on the training data, to balance our model to get good performance on both training and validation data.
And finally, the test data is totally un-used in our development of our model. And its performance is generally fair enough to compare different models.



In [0]:
test_datagen = datagen.flow_from_directory(directory=test_dir, 
                                                           shuffle=False, 
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150) 
                                                           batch_size=BATCH_SIZE, 
                                                           class_mode='binary')

score = model.evaluate_generator(generator=test_datagen,
                                 steps = int(np.ceil(total_test / float(BATCH_SIZE))))
print("Test Loss: ", score[0], "Test Accuracy: ", score[1])

## [Save and Load Model]

In [0]:
from keras.models import load_model

model.save('my_model.h5') 

# del model    # deletes the existing model

# model = tf.keras.models.load_model('my_model.h5')